In [ ]:
import numpy as np
from tqdm import tqdm
import math
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import argparse
import pandas as pd
from tensorflow.compat.v1.distributions import Bernoulli
from tensorflow.compat.v1.train import RMSPropOptimizer
import matplotlib.pyplot as plt
from scipy.sparse import save_npz, load_npz
from scipy.sparse import vstack, hstack

In [ ]:
#load and save files using the file name and path specified
def load_numpy(path, name):
    return load_npz(path+name).tocsr()

def load_dataframe_csv(path, name):
    return pd.read_csv(path+name)
    
def save_dataframe_csv(df, path, name):
    df.to_csv(path+name, index=False)

# VAE-CF Model


In [ ]:
#make predictions based on the embbeding matrices
def predict(matrix_U, matrix_V, topK, matrix_Train, bias, item_based=False):
    bias_vector = bias.copy()
    prediction = []

    for user_index in tqdm(range(matrix_U.shape[0])):
        vector_u = matrix_U[user_index]
        vector_train = matrix_Train[user_index]
        if item_based:   #user based, get single bias everytime for each user 
            bias = bias_vector[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            train_index = vector_train.nonzero()[1]
            predictions = matrix_V.dot(vector_u)
            if bias is not None:
                vector_predict = predictions + bias

        else:
            vector_predict = np.zeros(matrix_Train.shape[1], dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)

In [ ]:
#metrics
# Recall @ R
def recallk(vector_true_dense, hits):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

#support function for ndcg@k
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    return 0.

# ndcg@k
def ndcg_at_k(hits, k=100, **unused):
    """
    r: Relevance scores (list or numpy) in rank order
    """
    dcg_max = dcg_at_k(sorted(hits, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(hits, k) / dcg_max

In [ ]:
# evaluate the two metrics
def evaluate(matrix_Predict, matrix_Test, atK, metric_names= ['Recall', 'NDCG@K']):
    #mapping of metrics to functions
    local_metrics = {
        "Recall": recallk,
        "NDCG@K": ndcg_at_k
    }

    output = {}

    for k in atK:
        results = {name: [] for name in metric_names}
        #retrieve top k predictions
        topK_Predictions = matrix_Predict[:, :k]

        for user_index in range(topK_Predictions.shape[0]):
            predicted = topK_Predictions[user_index]
            if len(predicted.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_dense = vector_true.nonzero()[1]
                hits = np.isin(predicted, vector_dense)

                if vector_dense.size > 0:
                    for name in metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_dense, hits=hits))
        #create a summary of results
        summary = {}
        for name in metric_names:
            summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),1.96*np.std(results[name])/np.sqrt(matrix_Predict.shape[0]))
        output.update(summary)

    return output

In [ ]:
class VAE(object):

    def __init__(self, observation_dim, latent_dim, batch_size,
                 lamb=0.01,optimizer=RMSPropOptimizer):

        self._lamb = lamb
        self._beta = 0.2
        self._latent_dim = latent_dim
        self._batch_size = batch_size
        self._observation_dim = observation_dim
        print('observation dimension:', self._observation_dim)
        self._optimizer = optimizer
        self._observation_distribution = "Multinomial"
        self._observation_std = 0.01
        self._build_graph()

    def _build_graph(self):

        with tf.variable_scope('vae'):
            # create input as a column vector with the dimension of observations (item for user-based)
            self.input = tf.placeholder(tf.float32, shape=[None, self._observation_dim])
            # dropout rate of x are set to 0. The remaining elements are scaled up by 1.0 / (1 - rate)
            self.corruption = tf.placeholder(tf.float32)    # some corruption number
            self.sampling = tf.placeholder(tf.bool)         
            
            # adding a dropout layer 
            mask1 = tf.nn.dropout(tf.ones_like(self.input), 1 - self.corruption)

            input_dropout = self.input * mask1

            with tf.variable_scope('encoder'):
                # encoded weight for mean and vairance with I * 2K or U * 2K (user vs. item based)
                # each K for e
                encode_weights = tf.Variable(tf.truncated_normal([self._observation_dim, self._latent_dim*2],
                                                                 stddev=1 / 500.0), name="Weights")    ### !Hard-coded stdev! 
                # encoded biases, diagonal 
                encode_bias = tf.Variable(tf.constant(0., shape=[self._latent_dim*2]), name="Bias")

                # U * I @ I * 2K + ,2K  into the encoded space 
                encoded = tf.matmul(input_dropout, encode_weights) + encode_bias

            # gets the embedded z vector 
            with tf.variable_scope('latent'):
                # get encoded mean 
                self.mean = tf.nn.relu(encoded[:, :self._latent_dim])
                 # get standard deviation for latent 
                logstd = encoded[:, self._latent_dim:]
                self.stddev = tf.exp(logstd)
                # for parameterization trick 
                epsilon = tf.random_normal(tf.shape(self.stddev))
                
                self.z = tf.cond(self.sampling, lambda: self.mean + self.stddev * epsilon, lambda: self.mean)

            with tf.variable_scope('decoder'):

                self.decode_weights = tf.Variable(
                    tf.truncated_normal([self._latent_dim, self._observation_dim], stddev=1 / 500.0),
                    name="Weights")
                self.decode_bias = tf.Variable(tf.constant(0., shape=[self._observation_dim]), name="Bias")
                decoded = tf.matmul(self.z, self.decode_weights) + self.decode_bias

                self.obs_mean = decoded

                self.sample = Bernoulli(probs=self.obs_mean).sample()

            # identification of loss functions 
            with tf.variable_scope('loss'):
                # KL divergence
                with tf.variable_scope('kl-divergence'):
                    kl = self._kl_diagnormal_stdnormal(self.mean, logstd)

                # multinomial log likelihood 
                with tf.variable_scope('multinomial'):
                        obj = self._multinomial_log_likelihood(self.input, self.obs_mean)
                # l2 regularization 
                with tf.variable_scope('l2'):
                    l2_loss = tf.reduce_mean(tf.nn.l2_loss(encode_weights) + tf.nn.l2_loss(self.decode_weights))

                self._loss = self._beta * kl + obj + self._lamb * l2_loss

            with tf.variable_scope('optimizer'):
                optimizer = self._optimizer(learning_rate=1e-4)
            with tf.variable_scope('training-step'):
                self._train = optimizer.minimize(self._loss)

            self.sess = tf.Session()
            init = tf.global_variables_initializer()
            self.sess.run(init)

    @staticmethod
    def _kl_diagnormal_stdnormal(mu, log_std):
        var_square = tf.exp(2 * log_std)
        return 0.5 * tf.reduce_mean(tf.square(mu) + var_square - 1. - 2 * log_std)

    @staticmethod
    def _multinomial_log_likelihood(target, outputs, eps=1e-8):
        log_softmax_output = tf.nn.log_softmax(outputs)
        log_like = -tf.reduce_mean(tf.reduce_sum(log_softmax_output * target, axis=1))
        return log_like

    # take UI if user-based, or else IU 
    def train_model(self, rating_matrix, corruption, epoch=100):
        batches = self.get_batches(rating_matrix, self._batch_size)

        # Training
        for i in tqdm(range(epoch)):
            for step in range(len(batches)):
                feed_dict = {self.input: batches[step].todense(), self.corruption: corruption, self.sampling: True}
                training = self.sess.run([self._train], feed_dict=feed_dict)

    def get_batches(self, rating_matrix, batch_size):
        remaining_size = rating_matrix.shape[0]
        batch_index = 0
        batches = []
        while remaining_size > 0:
            if remaining_size < batch_size:
                batches.append(rating_matrix[batch_index*batch_size:])
            else:
                batches.append(rating_matrix[batch_index*batch_size:(batch_index+1)*batch_size])
            batch_index += 1
            remaining_size -= batch_size
        return batches

    # get embeddings
    def get_RQ(self, rating_matrix):
        batches = self.get_batches(rating_matrix, self._batch_size)
        RQ = []
        for step in range(len(batches)):
            feed_dict = {self.input: batches[step].todense(), self.corruption: 0, self.sampling: False}
            embedding = self.sess.run(self.z, feed_dict=feed_dict)
            RQ.append(embedding)

        return np.vstack(RQ)

    def get_Y(self):
        return self.sess.run(self.decode_weights)

    def get_Bias(self):
        return self.sess.run(self.decode_bias)

In [ ]:
def vae_cf(matrix_train, iteration=100, lam=80, rank=200, corruption=0.5):
    # record number of items for user-based, else number of users as _observation_dim
    m, n = matrix_train.shape

    #we assume observation are sampled from multinomial distribution 
    model = VAE(n, rank, 100, lamb=lam, optimizer=RMSPropOptimizer) # default batch_size as 100 here, 

    #train model
    model.train_model(matrix_train, corruption, iteration)

    #get model parameters to be returned
    RQ = model.get_RQ(matrix_train)
    Y = model.get_Y()
    Bias = model.get_Bias()

    #close tf session to avoid training more models on the same graph
    model.sess.close()
    tf.reset_default_graph()

    return RQ, Y, Bias

In [ ]:
def main(args):
    # Show hyper parameter settings
    print("\n___________parameter settings____________")
    print("Data Path: {0}".format(args.path))
    print("Rank: {0}".format(args.rank))
    print("Lambda: {0}".format(args.lamb))
    print("Number of iterations: {0}".format(args.iter))
    print("Evaluation - Topk: {0}".format(args.topk))

    # Load Data
    print("\n___________load data____________")
    R_train = load_numpy(path=args.path, name='Rtrain.npz')
    R_valid = load_numpy(path=args.path, name='Rvalid.npz')
    R_train = R_train + R_valid
    print("Train U-I Dimensions: {0}".format(R_train.shape))

    """User-based entry:"""
    print("\n___________User-based VAE-CF computation:____________")
    # RQ has dimension U * K
    # Y has dimension I * K
    # Bias has dimension (I,)
    RQ, Yt, Bias = vae_cf(R_train, iteration=args.iter, rank=args.rank,
                                      corruption=args.corruption,
                                      lam=args.lamb)
    Y = Yt.T
    
    print('RQ shape:', RQ.shape)
    print('Y shape', Y.shape)
    print('Bias shape:', Bias.shape)

    print("\n___________predict for user-based VAE:____________")
    user_based_prediction = predict(matrix_U=RQ,
                            matrix_V=Y,
                            bias=Bias,
                            topK=args.topk,
                            matrix_Train=R_train,
                            item_based = False)
    
    print(user_based_prediction.shape)

    """Item-based entry"""
    print("\n___________Item-based VAE-CF computation:____________")
    # Y has dimension I * K
    # RQ has dimension U * K
    # Bias has dimension (U,)
    Y, RQt, Bias = vae_cf(R_train.T, iteration=args.iter, rank=args.rank,
                                      corruption=args.corruption,
                                      lam=args.lamb)
    RQ = RQt.T
    
    print('RQ shape:', RQ.shape)
    print('Y shape', Y.shape)
    print('Bias shape:', Bias.shape)

    print("\n___________predict for item-based VAE:____________")
    item_based_prediction = predict(matrix_U=RQ,
                            matrix_V=Y,
                            bias=Bias,
                            topK=args.topk,
                            matrix_Train=R_train,
                            item_based = True)

    print(item_based_prediction.shape)

    print("\n___________weighted  user-item-based VAE:____________")
    # combine user and item based predictions
    UI_user_100_normalized = (user_based_prediction - np.mean(user_based_prediction)) / np.std(user_based_prediction)
    UI_item_100_normalized = (item_based_prediction - np.mean(item_based_prediction)) / np.std(item_based_prediction)
    weighted_UI = args.weight_u * UI_user_100_normalized + args.weight_i * UI_item_100_normalized

    UI_prediction = []
    # create top k predictions
    for user_index in tqdm(range(R_train.shape[0])):
        # get user training data 
        vector_train = R_train[user_index]
        train_index = vector_train.nonzero()[1]
        vector_predict = weighted_UI[user_index]

        # sort candidate items with descending predicted score 
        candidate_index = np.argpartition(-vector_predict, \
                                          args.topk+len(train_index))[:args.topk+len(train_index)]

        # get only the topk items and delete the ones inside the training dataset
        vector_predict = candidate_index[vector_predict[candidate_index].argsort()[::-1]]
        vector_predict = np.delete(vector_predict, np.isin(vector_predict, train_index).nonzero()[0])
        UI_prediction.append(vector_predict[:args.topk])
    
    UI_prediction = np.array(UI_prediction)
    print(UI_prediction.shape)

    print("\n___________create metrics____________")

    metric_names = ['Recall', 'NDCG@K']
    R_valid = load_numpy(path=args.path, name='Rtest.npz')
    result = evaluate(UI_prediction, R_valid, metric_names, [args.topk])
    print("\n")
    for metric in result.keys():
        print("{0}:{1}".format(metric, result[metric]))

In [ ]:
def main(args):
    # Show hyper parameter settings
    print("\n___________parameter settings____________")
    print("Data Path: {0}".format(args.path))
    print("Rank: {0}".format(args.rank))
    print("Lambda: {0}".format(args.lamb))
    print("Number of iterations: {0}".format(args.iter))
    print("Evaluation - Topk: {0}".format(args.topk))

    # Load Data
    print("\n___________load data____________")
    R_train = load_numpy(path=args.path, name='Rtrain.npz')
    R_valid = load_numpy(path=args.path, name='Rvalid.npz')
    R_train = R_train + R_valid
    print("Train U-I Dimensions: {0}".format(R_train.shape))

    """User-based entry:"""
    print("\n___________User-based VAE-CF computation:____________")
    # RQ has dimension U * K
    # Y has dimension I * K
    # Bias has dimension (I,)
    RQ, Yt, Bias = vae_cf(R_train, iteration=args.iter, rank=args.rank,
                                      corruption=args.corruption,
                                      lam=args.lamb)
    Y = Yt.T
    
    print('RQ shape:', RQ.shape)
    print('Y shape', Y.shape)
    print('Bias shape:', Bias.shape)

    print("\n___________predict for user-based VAE:____________")
    user_based_prediction = predict(matrix_U=RQ,
                            matrix_V=Y,
                            bias=Bias,
                            topK=args.topk,
                            matrix_Train=R_train,
                            item_based = False)
    
    print(user_based_prediction.shape)

    """Item-based entry"""
    print("\n___________Item-based VAE-CF computation:____________")
    # Y has dimension I * K
    # RQ has dimension U * K
    # Bias has dimension (U,)
    Y, RQt, Bias = vae_cf(R_train.T, iteration=args.iter, rank=args.rank,
                                      corruption=args.corruption,
                                      lam=args.lamb)
    RQ = RQt.T
    
    print('RQ shape:', RQ.shape)
    print('Y shape', Y.shape)
    print('Bias shape:', Bias.shape)

    print("\n___________predict for item-based VAE:____________")
    item_based_prediction = predict(matrix_U=RQ,
                            matrix_V=Y,
                            bias=Bias,
                            topK=args.topk,
                            matrix_Train=R_train,
                            item_based = True)

    print(item_based_prediction.shape)

    print("\n___________weighted  user-item-based VAE:____________")
    # combine user and item based predictions
    UI_user_100_normalized = (user_based_prediction - np.mean(user_based_prediction)) / np.std(user_based_prediction)
    UI_item_100_normalized = (item_based_prediction - np.mean(item_based_prediction)) / np.std(item_based_prediction)
    weighted_UI = args.weight_u * UI_user_100_normalized + args.weight_i * UI_item_100_normalized

    UI_prediction = []
    # create top k predictions
    for user_index in tqdm(range(R_train.shape[0])):
        # get user training data 
        vector_train = R_train[user_index]
        train_index = vector_train.nonzero()[1]
        vector_predict = weighted_UI[user_index]

        # sort candidate items with descending predicted score 
        candidate_index = np.argpartition(-vector_predict, \
                                          args.topk+len(train_index))[:args.topk+len(train_index)]

        # get only the topk items and delete the ones inside the training dataset
        vector_predict = candidate_index[vector_predict[candidate_index].argsort()[::-1]]
        vector_predict = np.delete(vector_predict, np.isin(vector_predict, train_index).nonzero()[0])
        UI_prediction.append(vector_predict[:args.topk])
    
    print(np.array(UI_prediction).shape)

    print("\n___________create metrics____________")
    R_test = load_numpy(path=args.path, name='Rtest.npz')
    result = evaluate(np.array(UI_prediction), R_test, [args.topk])
    print("\n")
    for metric in result.keys():
        print("{0}:{1}".format(metric, result[metric]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Tina
#data_path = '/content/drive/MyDrive/MASc_First_Year/CSC2515/CSC2515_Project_Part2/data/npz_files_threshold_4/'

#Sophie
data_path = '/content/drive/MyDrive/CSC2515_Project_Part2/data/npz_files_threshold_4/'

In [ ]:
best_hyper = {'NDCG': '[0.245, 0.0111]',
              'Recall@K': '[0.3625, 0.0169]',
              'corruption': 0.2,
              'iter': 200,
              'lambda': 0.0001,
              'rank': 100,
              'tok': 50.0,
              'weight_i': 0.1,
              'weight_u': 0.9}

In [ ]:
#test run
if __name__ == "__main__":
    # Commandline arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('-i', dest='iter', type=int, default=200)
    parser.add_argument('-l', dest='lamb', type=float, default=0.0001)
    parser.add_argument('-r', dest='rank', type=int, default=100)         # latent dimension
    parser.add_argument('-f')
    parser.add_argument('-c', dest='corruption', type=float, default=0.2)    #default dropout rate
    parser.add_argument('-d', dest='path', default=data_path)
    parser.add_argument('-k', dest='topk', type=int, default=50)
    parser.add_argument('-weight_u', dest='weight_u', type=float, default=0.9)
    parser.add_argument('-weight_i', dest='weight_i', type=float, default=0.1)
    args = parser.parse_args()

    main(args)


___________parameter settings____________
Data Path: /content/drive/MyDrive/CSC2515_Project_Part2/data/npz_files_threshold_4/
Rank: 100
Lambda: 0.0001
Number of iterations: 100
Evaluation - Topk: 50

___________load data____________
Train U-I Dimensions: (943, 1682)

___________User-based VAE-CF computation:____________
observation dimension: 1682


 18%|█▊        | 173/943 [00:00<00:00, 1634.46it/s]

RQ shape: (943, 100)
Y shape (1682, 100)
Bias shape: (1682,)

___________predict for user-based VAE:____________


100%|██████████| 943/943 [00:00<00:00, 1800.15it/s]


(943, 1682)

___________Item-based VAE-CF computation:____________
observation dimension: 943


...

(943, 1682)

___________weighted  user-item-based VAE:____________


 40%|████      | 378/943 [00:00<00:00, 3775.52it/s]

(943, 50)

___________create metrics____________


100%|██████████| 943/943 [00:00<00:00, 1652.83it/s]



Recall@50:(0.3269228351037726, 0.014849081519205541)
NDCG@K:(0.4724168546173931, 0.014894197793072874)


# Hyperparameter Tuning

In [ ]:
def hyper_parameter_tuning(R_train, R_valid, params, save_path):
  # df = pd.read_csv(save_path+'hyperparam_continued.csv')
  df = pd.DataFrame(columns=list(params.keys()))

  print("Train U-I Dimensions: {0}".format(R_train.shape))

  for rank in params['rank']:
    for lam in params['lambda']:
      for corruption in params['corruption']:
        for topK in params['topK']:
          for weight_u_i in params['weight_u_i']:

            print("rank: {0}, lambda: {1}, corruption: {2}, topK: {3}, wegith_u_i: {4}"\
                              .format(rank, lam, corruption, topK, weight_u_i))
            
            RQ, Yt, Bias = vae_cf(R_train, iteration=params['iter'], rank=rank,
                                              corruption=corruption,
                                              lam=lam)
            Y = Yt.T
            user_based_prediction = predict(matrix_U=RQ,
                                    matrix_V=Y,
                                    bias=Bias,
                                    topK=topK,
                                    matrix_Train=R_train,
                                    item_based = False)

            Y, RQt, Bias = vae_cf(R_train.T, 
                                iteration=params['iter'], rank=rank,
                                corruption=corruption,lam=lam)
            RQ = RQt.T
            item_based_prediction = predict(matrix_U=RQ,
                                    matrix_V=Y, bias=Bias,
                                    topK=topK, matrix_Train=R_train,
                                    item_based = True)

            # combine user and item based predictions
            UI_user_100_normalized = (user_based_prediction - np.mean(user_based_prediction)) / np.std(user_based_prediction)
            UI_item_100_normalized = (item_based_prediction - np.mean(item_based_prediction)) / np.std(item_based_prediction)
            weighted_UI = weight_u_i[0] * UI_user_100_normalized + weight_u_i[1] * UI_item_100_normalized

            UI_prediction = []
            # create top k predictions
            for user_index in tqdm(range(R_train.shape[0])):
                # get user training data 
                vector_train = R_train[user_index]
                train_index = vector_train.nonzero()[1]
                vector_predict = weighted_UI[user_index]

                # sort candidate items with descending predicted score 
                candidate_index = np.argpartition(-vector_predict, topK+len(train_index))[:topK+len(train_index)]
                # get only the topk items and delete the ones inside the training dataset
                vector_predict = candidate_index[vector_predict[candidate_index].argsort()[::-1]]
                vector_predict = np.delete(vector_predict, np.isin(vector_predict, train_index).nonzero()[0])
                UI_prediction.append(vector_predict[:topK])
            
            result = evaluate(np.array(UI_prediction), R_valid, params['metric'], [topK])
            
            result_dict = {'rank': rank, 'lambda': lam, 'topK': topK,\
                            'iter': params['iter'], 'similarity': params['similarity'],\
                            'corruption': corruption, 'weight_u': weight_u_i[0], \
                            'weight_i': weight_u_i[1]}

            # add in metric results
            for name in result.keys():
                result_dict[name] = [round(result[name][0], 4), round(result[name][1], 4)]
            df = df.append(result_dict, ignore_index=True)
            save_dataframe_csv(df, save_path, 'hyperparams_tunning.csv')

In [ ]:
def main(args):
    params = {'corruption': [0.2, 0.3, 0.4, 0.5], \
              'rank': [50, 100], 'topK': [50,100], \
              'iter': 200, 'lambda': [0.00001, 0.0001, 0.001], \
              'weight_u_i':[(1,0), (0.9,0.1),(0.8,0.2),(0.7,0.3),(0.6,0.4),(0.5,0.5),\
                            (0.4,0.6), (0.3,0.7), (0.2,0.8),(0.1,0.9),(0,1)],\
              'metric': ['NDCG@K', 'Recall']}

    # load training and testing data 
    R_train = load_numpy(path=args.path, name='Rtrain.npz')
    R_valid = load_numpy(path=args.path, name='Rvalid.npz')
    
    # merge training and validation data 
    R_train = R_train + R_valid
    R_test = load_numpy(path=args.path, name='Rtest.npz')

    # call hyperparameter tuning method
    hyper_parameter_tuning(R_train, R_test, params, save_path=args.hyper_path)

if __name__ == "__main__":
    hyperparameter_folder = '/content/drive/MyDrive/MASc_First_Year/CSC2515/CSC2515_Project_Part2/hyperparameter_tuning_thresholded/'

    # Commandline arguments
    parser = argparse.ArgumentParser(description="ParameterTuning")
    parser.add_argument('-d', dest='path', default=data_path)
    parser.add_argument('-hyper_path', dest='hyper_path', default=hyperparameter_folder)
    parser.add_argument('-f')
    args = parser.parse_args()

    main(args)

In [ ]:
final_UI_prediction = np.load('results/final_UI100.npy')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MASc_First_Year/CSC2515/CSC2515_Project_Part2/hyperparameter_tuning/hyperparam.csv')